In [18]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, brier_score_loss, log_loss
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, make_scorer, roc_curve, auc
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, learning_curve, ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.utils import resample
import xgboost as xgb
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# Build a random_grid for going over hyperparameters
def opt_random_forest(X_train,y_train,X_test,y_test,labels):
    
    
    scorer = make_scorer(precision_score)

    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 10, stop = 500, num = 3)]
    # Minimum number of samples required to split a node
    min_samples_split = [2, 4]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap};

    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestClassifier(random_state = 42);
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = GridSearchCV(estimator = rf, 
                             param_grid = random_grid, 
                             scoring=scorer,
                             cv = 3, 
                             verbose=2, 
                             n_jobs = -1);
    # Fit the random search model

    rf_random.fit(X_train, y_train);

    base_model = RandomForestClassifier(n_estimators = 10, random_state = 42);
    pl = 1
    which_model = "Base Random Forest"
    base_model.fit(X_train, y_train);
    base_accuracy = evaluate(base_model, X_test, y_test,labels,pl,which_model)
    

    print(rf_random.best_params_)
    best_random = rf_random.best_estimator_
    which_model = "Optimized Random Forest"
    random_accuracy = evaluate(best_random, X_test, y_test,labels, pl, which_model)

In [20]:
    
def opt_log_reg(X_train,y_train,X_test,y_test,labels):
    # Build a random_grid for going over hyperparameters
    scorer = make_scorer(roc_auc_score)


    # Create regularization penalty space
    penalty = ['l1']

    # Create regularization hyperparameter distribution using uniform distribution
    C = np.array(np.linspace(0.5,1,10))

    # Create hyperparameter options
    random_grid = dict(C=C, penalty=penalty);


    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    lg = LogisticRegression(solver ='liblinear', random_state=42);
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    lg_random = GridSearchCV(estimator = lg, 
                                   param_grid = random_grid,
                                   scoring = scorer,
                                   cv = 3, 
                                   verbose=2, 
                                   n_jobs = -1);

    # Fit the random search model
    lg_random.fit(X_train, y_train);

    lg_random.best_params_

    base_model = LogisticRegression(random_state = 42, solver ='liblinear');
    pl = 1
    which_model = "Base Logistic Regression"
    base_model.fit(X_train, y_train);
    base_accuracy = evaluate(base_model, X_test, y_test,labels,pl, which_model)

    print(lg_random.best_params_)
    best_random = lg_random.best_estimator_
    which_model = "Optimized Logistic Regression"
    random_accuracy = evaluate(best_random, X_test, y_test,labels, pl, which_model)
    

In [21]:
def opt_xgboost(X_train,y_train,X_test,y_test,labels):
    # Create XGB Classifier object

    scorer = make_scorer(precision_score);
    xgb_clf = xgb.XGBClassifier(tree_method = "auto", predictor = "cpu_predictor", verbosity = 0,
                               eval_metric = ["error","map", "auc"], objective = "binary:logistic", seed =42);
    # Create parameter grid
    parameters = {"learning_rate": [0.1, 0.01, 0.001],
                   "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
                   "max_depth": [2, 4, 7, 10],
                   "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
                   "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
                   "reg_alpha": [0, 0.5, 1],
                   "reg_lambda": [1, 1.5, 2, 3, 4.5],
                   "min_child_weight": [1, 3, 5, 7],
                   "n_estimators": [100, 250, 500, 1000]};

    # Create RandomizedSearchCV Object
    xgb_rscv = RandomizedSearchCV(xgb_clf, 
                                  param_distributions = parameters, 
                                  scoring=scorer,
                                  cv = 3, 
                                  verbose = 1, 
                                  random_state = 42);

    # Fit the model
    pl = 1

    model_xgboost = xgb_rscv.fit(X_train, y_train);

    print(model_xgboost.best_params_)

    best_random_xgb = model_xgboost.best_estimator_
    which_model = "XGBoost"
    accuracy = evaluate(best_random_xgb, X_test, y_test,labels, pl, which_model)

In [22]:
def assign_labels(binary,labels):
    labelled = [labels[x] for x in binary]
    return labelled

def evaluate(model, X_test, y_test, labels,pl, which_model):
    try:
        print("Model coefficients:",model.coef_)
    except:
        print("Feature importances:", model.feature_importances_)
    
    predictions = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[::,1]
    fig = plt.figure(figsize=(12,4))
    y_axis_labels = ['Test','Predictions','Probabilities']
    sns.heatmap([y_test,predictions,y_prob], square=True, 
                                             cmap='plasma',
                                             cbar=False, 
                                             yticklabels=y_axis_labels)
    test_label = assign_labels(y_test,labels)
    pred_label = assign_labels(predictions,labels)
    confusion_mat = confusion_matrix(test_label,pred_label)

    print(classification_report(y_test, predictions, target_names=labels))

    fpr, tpr, thresholds = roc_curve(y_test,y_prob,pos_label=pl)
    AUC = auc(fpr, tpr)
    print('Confusion matrix: ')
    print('Predicted')
    print("     ",labels)
    print("Test ",labels[0],confusion_mat[0])
    print("     ",labels[1],confusion_mat[1])
    
    result_table = {'fpr':fpr, 'tpr':tpr, 'auc':AUC}

    fig = plt.figure(figsize=(8,6))
    
    plt.plot(result_table['fpr'], 
             result_table['tpr'], 
             label="AUC={:.3f}".format(result_table['auc']))
    
    plt.plot([0,1], [0,1], color='black', linestyle='--')

    plt.xticks(np.arange(0.0, 1.1, step=0.1))
    plt.xlabel("False Positive Rate", fontsize=15)

    plt.yticks(np.arange(0.0, 1.1, step=0.1))
    plt.ylabel("True Positive Rate", fontsize=15)

    plt.title('ROC Curve Analysis for '+which_model, fontweight='bold', fontsize=15)
    plt.legend(prop={'size':13}, loc='lower right')

    plt.show()
    
    return result_table

In [23]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    From: https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("F1 Score")
    scorer = make_scorer(f1_score)
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=scorer)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [24]:
def abundance_comparison(df, IDs,norm):
    
    df = df[IDs+['Group']]
    df_long = pd.melt(df, "Group", var_name="Taxon", value_name="abundance")
    df_long = df_long[~(df_long == 0).any(axis=1)]
    if norm=='clr':
        df_long['abundance'] = df_long['abundance'].apply(lambda x: math.exp(x))

    plt.figure(figsize=(16, 6))
    sns.boxplot(x="Taxon", y="abundance",
            hue="Group",
            data=df_long)
    #plt.ylim(0, 25000)

    for i in IDs:
        grouped = df.groupby("Group")[i]
        value_dict ={}
        dists = []
        for key, item in grouped:
            dists.append(grouped.get_group(key).values)
        print("For OSU ID {} , t-test statistic: ".format(i),
              stats.ttest_ind(dists[0], dists[1]))